# Models
Models will have a common superclass and three methods, one for data collection, the main algorithm, and relevant data used in that algorithm.
There are three types of models, macro, asset-specific, and opportunity-specific. Macro models look at the global environment, asset-specific models find opportunities within one asset class, and opportunity-specific models locate them across multiple asset classes

Here is the `Model` superclass:

In [1]:
from abc import ABC, abstractmethod
class Model(ABC):
    def __init__(self):
        self.get_data()
    
    @abstractmethod
    # load the data into the model
    def get_data(self):
        pass
    @abstractmethod
    # run algorithm and save output
    def analyze(self, refresh=False):
        pass
    @abstractmethod
    # return some data to help understand analysis
    def relevant_data(self):
        pass
    
    

#### Helper Classes & Methods

In [2]:
import quandl
quandl.ApiConfig.api_key = 'yACpi8J95VWoPd_8qYmv'
from alpha_vantage.timeseries import TimeSeries
ts = TimeSeries(key='2MI58JUQIRZ6I9AA', output_format='pandas', indexing_type='date')

import pandas as pd
import numpy as np
import logging
logging.basicConfig(level=logging.INFO)

In [3]:
# %run Helpers.ipynb
from helpers import *

In [4]:
# determine asset tilt at the end of analysis
class AssetTilt:
    MACRO = []
    # [developed stocks, EM stocks, IRs, developed currencies, EM currencies, energy, agriculture, precious metals, industrial metals]
#     TILT = [[], [], [], [], [], [], [], [], []]
    
#     @staticmethod
#     def get_tilt():
#         tilt = []
#         for i in AssetTilt.TILT:
#             cnt = len(i)
#             if i:
#                 tilt.append((i.count(1) / cnt) - (i.count(0) / cnt))
#             else:
#                 tilt.append(0)
        
#         return tilt

# holds list of all recommendations
class Recommendations:
    RCMDS = []
    
    @classmethod
    def output(cls):
        output = []
        for r in cls.RCMDS:
            output.append([r.r_type, r.asset, r.analysis])
        return output

LOGS = []

## Macro Models
These models assess the global macro environment and determine asset tilt and the base for further analysis. They are also used to create a beta underlay in an investment portfolio.

In [5]:
class EconomicCycleModel(Model):
    def __init__(self):
        super().__init__()
    
    def get_data(self):
        # get data from quandl
        self.retail_chg = quandl.get(Data.RETAIL, start_date=years_ago(6), end_date=date_fmt(data_date)).pct_change(periods=12).dropna()
        self.retail = self.retail_chg.diff(periods=12).rolling(window=12).mean().dropna()
        self.monsply_chg = quandl.get(Data.MONSPLY, start_date=years_ago(5), end_date=date_fmt(data_date)).pct_change(periods=52).dropna()
        self.monsply = self.monsply_chg.pct_change(periods=52).dropna()
        self.businv_chg = quandl.get(Data.BUSINV, start_date=years_ago(5), end_date=date_fmt(data_date)).pct_change(periods=12).dropna()
        self.businv = self.businv_chg.diff(periods=12).dropna()
        self.loans_chg = quandl.get(Data.LOANS, start_date=years_ago(4), end_date=date_fmt(data_date)).pct_change(periods=12).dropna()
        self.loans = self.loans_chg.diff(periods=12).dropna()
        self.ff = quandl.get(Data.FF, start_date = years_ago(3), end_date=date_fmt(data_date)).diff(periods=52).dropna()
        self.ff_chg = self.ff.diff(periods=6).dropna()
        
        self.ngdp = quandl.get(Data.NGDP, start_date=years_ago(3), end_date=date_fmt(data_date))
        self.pgdp = quandl.get(Data.PGDP, start_date=years_ago(3), end_date=date_fmt(data_date))
        self.gdp_gap = self.ngdp.div(self.pgdp).sub(1).dropna()
        
        # get last value
        self.l_r = last(self.retail)
        self.l_rg = last(self.retail_chg)
        self.l_m = last(self.monsply)
        self.l_mg = last(self.monsply_chg)
        self.l_b = last(self.businv)
        self.l_bg = last(self.businv_chg)
        self.l_l = last(self.loans)
        self.l_lg = last(self.loans_chg)
        self.l_f = last(self.ff)
        self.l_fg = last(self.ff_chg)
        self.l_g = last(self.gdp_gap)
        
        # get means
        self.m_r = mean(self.retail)
        self.m_b = mean(self.businv)
        self.m_l = mean(self.loans)
        
        # get standard deviations
        self.s_r = std(self.retail)
        self.s_b = std(self.businv)
        self.s_l = std(self.loans)
        
    def analyze(self, refresh=False):
        if refresh:
            super().__init__()
        
        # early, mid, late, recession
        cycle = np.array([0, 0, 0, 0], dtype='f')
        num = 0
        
        # gdp gap
        if self.l_g > 0:
            LOGS.append('GDP gap positive')
            cycle[2] += 2
            num += 2
        if self.gdp_gap.gt(0).sum().sum() > 0 and self.l_g < 0:
            LOGS.append('GDP gap moved negative')
            cycle[3] += 1
            num += 1
        
        # retail sales
        if self.l_r > (self.m_r + (self.s_r / 2)) and self.retail.lt(self.m_r - (self.s_r /2)).sum().sum() == 0:
            LOGS.append('Retail sales above 0.5 std; haven\'t been below -0.5 std' )
            cycle[2] += 1
            num += 1
        if self.l_r < 0.02:
            LOGS.append('Change in % retail sales change less than 2%')
            cycle[3] += 1
            num += 1
        if self.l_rg < 0:
            LOGS.append('Change in retail sales negative')
            cycle[1] += 1
            num += 1
        if self.l_r < 0.25:
            cycle[0] += 1
            num += 1
        if self.retail.lt(self.m_r - (self.s_r /2)).sum().sum() > 0 and self.l_r > (self.m_r + (self.s_r / 2)):
            LOGS.append('Retail sales have been below 0.5 std; are now above 0.5 std' )
            cycle[0] += 1
            
        # business inventories
        if self.l_bg < 0:
            LOGS.append('Change in business inventories growing')
            cycle[1] += 1
            num += 1
        if self.l_b > 0 and self.l_b > (self.m_b + (self.s_b / 2)):
            LOGS.append('Change in business inventory growth greater than 0.5 std')
            cycle[2] += 1
            num += 1
        if self.businv.lt(self.m_b - (self.s_b /2)).sum().sum() > 0 and self.l_b > (self.m_b + (self.s_b / 2)):
            LOGS.append('Change in business inventory growth has been negative and is greater than 0.5 std')
            cycle[0] += 1
            
        # interest rates
        if self.l_f > 0.15:
            LOGS.append('1-year growth of FF rate greater than 15 bips')
            cycle[1] += 1
            num += 1
        if self.l_f < 0.5:
            LOGS.append('1-year growth of FF rate less than 50 bips')
            cycle[0] += 1
            num += 1
        if self.l_f < 0 and self.l_fg < 0:
            LOGS.append('FF rate declining')
            cycle[3] += 1
        
        # business loans
        if self.l_lg < 0:
            LOGS.append('Change in business loans down')
            cycle[1] += 1
            num += 1
        if self.l_l > 0 and self.l_l > (self.m_l + (self.s_l / 2)):
            LOGS.append('Business loans positive and above 0.5 std')
            cycle[2] += 1
            num += 1
            
        # money supply
        if self.monsply_chg.lt(0).sum().sum() > 0:
            LOGS.append('Money supply decreasing')
            cycle[2] += 1
            num += 1
            
        cycle = cycle / num
        
        
        cycle_name = {
            0: "Early-cycle",
            1: "Mid-cycle",
            2: "Late-cycle",
            3: "Recession",
        }
        cur_cycle = cycle.argmax(axis=0)
        AssetTilt.MACRO.append(cycle_name.get(cur_cycle))
            
        return cycle
        
    def relevant_data(self):
        return {
            "retail_sales": self.retail_chg,
            "money_supply": self.monsply_chg,
            "business_inventories": self.businv_chg,
            "business_loans": self.loans_chg,
            "fed_funds": self.ff
        }
    
class LiquidityModel(Model):
    def __init__(self):
        super().__init__()

    def get_data(self):
        self.anfci = quandl.get(Data.ANFCI, start_date=years_ago(3), end_date=date_fmt(data_date))
        self.leverage = quandl.get(Data.LEVERAGE, start_date=years_ago(3), end_date=date_fmt(data_date))
        self.ted = quandl.get(Data.TED, collapse='weekly', start_date=years_ago(3), end_date=date_fmt(data_date))
        self.yldcrv = quandl.get(Data.YLDCRV, start_date=years_ago(3), end_date=date_fmt(data_date))
        self.lo = quandl.get(Data.ED1, start_date=years_ago(3), end_date=date_fmt(data_date))
        
        self.l_a = last(self.anfci)
        self.c_a = last(self.anfci.pct_change(periods=52))
        
        self.l_l = last(self.leverage)
        self.c_l = last(self.leverage.pct_change(periods=52))
        
        self.l_t = last(self.ted)
        self.c_t = last(self.ted.pct_change(periods=52))
        
        self.l_y = last(self.yldcrv)

    def analyze(self, refresh=False):
        # strengthening, weakening
        self.change = np.array([0, 0], dtype='f')
        # strong, weak
        self.position = np.array([0, 0], dtype='f')
        
        if self.l_a > 0:
            self.position[1] += 1
        else:
            self.position[0] += 1
        if self.c_a > 0:
            self.change[1] += 1
        else:
            self.change[0] += 1
        
        if self.l_l > 0:
            self.position[1] += 1
        else:
            self.position[0] += 1
        if self.c_l > 0:
            self.change[1] += 1
        else:
            self.change[0] += 1   
            
        if self.l_t > 0.6:
            self.position[1] += 1
        else:
            self.position[0] += 1
        if self.c_t > 0:
            self.change[1] += 1
        else:
            self.change[0] += 1

        if self.l_y < 0:
            self.position[0] += 1
        else:
            self.position[1] += 1
            
        total_change = sum(self.change)
        total_position = sum(self.position)
        self.position /= total_position
        self.change /= total_change
        strong = self.position[0] > 0.5
        strengthening = self.change[0] > 0.5
        text = ''
        
        if strong:
            text += 'Strong '
        else:
            text += 'Weak '
            
        if strong == strengthening:
            text += 'and '
        else:
            text += 'but '
            
        if strengthening:
            text += 'strengthening '
        else:
            text += 'weakening '
            
        text += 'liquidity'
        
        AssetTilt.MACRO.append(text)
        
        return [self.position, self.change]

    def relevant_data(self):
        return {
            "anfci": self.anfci,
            "leverage": self.leverage,
            "ted": self.ted,
            "yldcrv": self.yldcrv
        }
    
class CorePeripheryModel(Model):
    def __init__(self):
        super().__init__()

    def get_data(self):
        self.dxy_orig = quandl.get(Data.DXY + ".4", start_date=years_ago(16), end_date=date_fmt(data_date)).rolling(window=100).mean().pct_change(periods=100)
        self.dxy_c_orig = self.dxy_orig.diff(periods=100)
        self.dxy = last(self.dxy_orig, col='Settle')
        self.dxy_c = last(self.dxy_c_orig, col='Settle')
        
    def analyze(self, refresh=False):
        if (self.dxy > 0 and self.dxy_c > 0):
            AssetTilt.MACRO.append('Money flowing to core')
            Recommendations.RCMDS.append(Recommendation(Recommendation.SPREAD, 'SPY/EEM', 'Buy US against EM, rush to safety'))
        elif (self.dxy < 0 and self.dxy_c < 0):
            AssetTilt.MACRO.append('Money flowing to periphery')
            Recommendations.RCMDS.append(Recommendation(Recommendation.SPREAD, 'EEM/SPY', 'Buy EM against US, follow trend'))

    def relevant_data(self):
        return {
            "dxy": self.dxy_orig,
            "dxy_chg": self.dxy_c_orig
        }

## Asset-Specific Models
These models look at specific assets and determine whether their is an oppurtunity available. They may get some information from the macro models above

In [6]:
class CapitalCycleModel(Model):
    def __init__(self):
        super().__init__()

    def get_data(self):
        self.inv = quandl.get(Data.OILINV, start_date=years_ago(15), end_date=date_fmt(data_date)).pct_change(periods=12).diff(periods=12)
        self.price = quandl.get('OPEC/ORB', start_date=years_ago(15), end_date=date_fmt(data_date))
        self.cap = quandl.get(Data.OILCAP, start_date=years_ago(15), end_date=date_fmt(data_date)).rolling(window=12).mean()
        
        self.l_i = last(self.inv)
        self.l_c = last(self.cap)
        
    def analyze(self, refresh=False):
        #TODO: Add recommendation
        if self.l_c > 90:
            LOGS.append('Oil capacity utilization high')
        if self.l_i > 0.15:
            LOGS.append('Oil inventories increasing over 15%')
        if self.l_c < 85:
            LOGS.append('Oil capacity utilization low')
        if self.l_i < 0:
            LOGS.append('Oil inventories decreasing')
            
    
    def relevant_data(self):
        return {
            "cap": self.cap,
            "inventories": self.inv
        }
    
class CurrencyModel(Model):
    def __init__(self):
        super().__init__()

    def get_data(self):
        pass
    def analyze(self, refresh=False):
        pass
    def relevant_data(self):
        pass
    
class RiskPremiumModel(Model):
    def __init__(self):
        super().__init__()

    def get_data(self):
        self.rp = quandl.get([Data.EY, Data.TBILL], start_date=years_ago(3), end_date=date_fmt(data_date))
        self.rp = (self.rp[self.rp.columns[0]] - self.rp[self.rp.columns[1]]).diff(periods=12).dropna()
        self.l_rp = self.rp.iloc[-1]

    def analyze(self, refresh=False):
        if self.l_rp > 1.5:
            LOGS.append('Earnings yields increasing against T-Bill rates')
        elif self.l_rp < -1.5:
            LOGS.append('Earnings yields decreasing against T-Bill rates')

    def relevant_data(self):
        return {
            "risk_premium": self.rp
        }

## Opportunity-Specific Models
These models locate opportunities based on rules that work across multiple asset classes.

In [7]:
class ReflexivityModel(Model):
    def __init__(self):
        super().__init__()

    def get_data(self):
        self.vix = quandl.get(Data.VIX + ".5", start_date=years_ago(5), end_date=date_fmt(data_date))
        self.vix_ma_all = self.vix.rolling(window=100).mean() - self.vix.rolling(window=50).mean()
        self.vix_ma = last(self.vix_ma_all, col='Settle')
        
        self.sent_all = quandl.get(Data.SENT + ".6", start_date=years_ago(5), end_date=date_fmt(data_date)).rolling(window=4).mean()
        self.sent = last(self.sent_all, col='Bull-Bear Spread')

    def analyze(self, refresh=False):
        if (self.vix_ma > -1 and self.vix_ma < 1 and last(self.vix, col='Settle') < 14):
            Recommendations.RCMDS.append(Recommendation(Recommendation.SELL, 'SPY', 'Extreme low volatility'))
        
        if (self.sent > 0.25):
            LOGS.append('Bull-Bear spread greater than 25%')
            Recommendations.RCMDS.append(Recommendation(Recommendation.SELL, 'SPY', 'Overly bullish sentiment'))
        elif (self.sent < -0.15):
            LOGS.append('Bull-Bear spread less than -15%')
            Recommendations.RCMDS.append(Recommendation(Recommendation.BUY, 'SPY', 'Overly bearish sentiment'))

    def relevant_data(self):
        return {
            "vix_ma": self.vix_ma_all,
            "sentiment": self.sent_all
        }
    
class BondMarketModel(Model):
    def __init__(self):
        super().__init__()

    def get_data(self):
        lqd, _ = ts.get_daily_adjusted(symbol='LQD', outputsize='full')
        ief, _ = ts.get_daily_adjusted(symbol='IEF', outputsize='full')
        spy_u, _ = ts.get_daily_adjusted(symbol='SPY', outputsize='full')

        self.data = lqd['4. close'] / ief['4. close']
        self.data = self.data.tail(300).pct_change(periods=90).dropna()

        self.spy = spy_u['4. close']
        self.spy = self.spy.tail(300).pct_change(periods=90).dropna()
        
        self.spy_mean = self.spy.values.mean()
        self.spy_std = self.spy.values.std(ddof=1)
        
        self.data_mean = self.data.values.mean()
        self.data_std = self.data.values.std(ddof=1)
        
        self.s = self.spy.iloc[-1]
        self.l = self.data.iloc[-1]

    def analyze(self, refresh=False):
        spy_quadrant = 0
        if (self.s < self.spy_mean - self.spy_std):
            spy_quadrant = 1
        elif (self.s < self.spy_mean - (self.spy_std / 2)):
            spy_quadrant = 2
        elif (self.s < self.spy_mean):
            spy_quadrant = 3
        elif (self.s < self.spy_mean + (self.spy_std / 2)):
            spy_quadrant = 4
        elif (self.s < self.spy_mean + self.spy_std):
            spy_quadrant = 5
        else:
            spy_quadrant = 6
            
        data_quadrant = 0
        if (self.l < self.data_mean - self.data_std):
            data_quadrant = 1
        elif (self.l < self.data_mean - (self.data_std / 2)):
            data_quadrant = 2
        elif (self.l < self.data_mean):
            data_quadrant = 3
        elif (self.l < self.data_mean + (self.data_std / 2)):
            data_quadrant = 4
        elif (self.l < self.data_mean + self.data_std):
            data_quadrant = 5
        else:
            data_quadrant = 6
        
        if (spy_quadrant > data_quadrant + 1):
            LOGS.append('SPY above LQD/IEF')
            Recommendations.RCMDS.append(Recommendation(Recommendation.SELL, 'SPY', 'SPY above LQD/IEF'))
        elif (spy_quadrant < data_quadrant - 1):
            LOGS.append('SPY below LQD/IEF')
            Recommendations.RCMDS.append(Recommendation(Recommendation.BUY, 'SPY', 'SPY below LQD/IEF'))
            
    
    def relevant_data(self):
        return {
            "spy_v_lqd/ief": pd.concat([self.data, self.spy], axis=1)
        }
    
class FalseConsensusModel(Model):
    def __init__(self):
        super().__init__()

    def get_data(self):
        self.data_fin = [] # financials
        self.data_com = [] # commodities
        for fin in COT.FIN:
            self.data_fin.append(COT.format_fin(quandl.get(COT.data(fin), start_date=years_ago(5), end_date=date_fmt(data_date)), True))
            
        for com in COT.COM:
            self.data_com.append(COT.format_com(quandl.get(COT.data(com), start_date=years_ago(5), end_date=date_fmt(data_date)), True))

    def analyze(self, refresh=False):
        self.extreme_fin = []
        self.extreme_com = []
        
        for n in range(len(self.data_fin)):
            #TODO: 3 currencies -> DXY
            if (self.data_fin[n] > .9):
                self.extreme_fin.append(COT.FIN[n][2])
                Recommendations.RCMDS.append(Recommendation(Recommendation.SELL, COT.FIN[n][2], 'Crowded long'))
            elif (self.data_fin[n] <.1):
                self.extreme_fin.append(COT.FIN[n][2])
                Recommendations.RCMDS.append(Recommendation(Recommendation.BUY, COT.FIN[n][2], 'Crowded short'))
                
        for n in range(len(self.data_com)):
            if (self.data_com[n] > .9):
                self.extreme_com.append(COT.COM[n][2])
                Recommendations.RCMDS.append(Recommendation(Recommendation.SELL, COT.COM[n][2], 'Crowded long'))
            elif (self.data_com[n] <.1):
                self.extreme_com.append(COT.COM[n][2])
                Recommendations.RCMDS.append(Recommendation(Recommendation.BUY, COT.COM[n][2], 'Crowded short'))
        
    def relevant_data(self):
        pass

#### TODO
- Recession + strengthening liquidity: cash pump -> buy
- BUY/SELL SPY + ES -> strong signal
- BUY/SELL 3 currencies -> strong signal (DXY)
- Oil cap, inv dec -> sell/buy future
- Weakening liquidity -> cash
- Commodity positioning + seasonality -> signal

In [ ]:
models = {
    'economic_cycle': EconomicCycleModel(),
    'liquidity': LiquidityModel(),
    'core_periphery': CorePeripheryModel(),
    'capital_cycle': CapitalCycleModel(),
#     'currency_cycle': CurrencyModel(),
#     'risk_premium': RiskPremiumModel(),
    'reflexivity': ReflexivityModel(),
    'bond_market': BondMarketModel(),
    'false_consensus': FalseConsensusModel()
}

def analyze_all():
    LOGS.clear()
    Recommendations.RCMDS.clear()
    AssetTilt.MACRO.clear()
    LOGS.append(str(data_date))
    for key, model in models.items():
        model.analyze()